In [14]:
import cv2
import os
import json
from mediapipe.python.solutions import hands as mp_hands
from mediapipe.python.solutions import pose as mp_pose
from mediapipe.python.solutions import drawing_utils as mp_drawing

# Türkçe karakterleri ASCII uyumlu hale getir
def normalize_filename(text):
    replacements = {
        "ç": "c", "ğ": "g", "ı": "i", "ö": "o", "ş": "s", "ü": "u",
        "Ç": "C", "Ğ": "G", "İ": "I", "Ö": "O", "Ş": "S", "Ü": "U"
    }
    for turkish_char, ascii_char in replacements.items():
        text = text.replace(turkish_char, ascii_char)
    return text

def process_video(video_path, label, json_data):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Video açılırken hata oluştu: {video_path}")
        return

    keypoints_sequence = []  # Zamansal seriyi kaydetmek için diziler
    frame_count = 0

    with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,  # İki eli algılamak için
        min_detection_confidence=0.5
    ) as hands, mp_pose.Pose(
        static_image_mode=False,
        model_complexity=1,
        enable_segmentation=False,
        min_detection_confidence=0.5
    ) as pose:

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # RGB formatına dönüştür
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # MediaPipe Pose ve Hands ile işlem yap
            hands_result = hands.process(image_rgb)
            pose_result = pose.process(image_rgb)

            # Tüm keypoint'leri birleştir
            frame_keypoints = []

            # El keypoints'lerini ekle
            if hands_result.multi_hand_landmarks:
                for hand_landmarks in hands_result.multi_hand_landmarks:
                    for lm in hand_landmarks.landmark:
                        frame_keypoints.extend([lm.x, lm.y, lm.z])  # X, Y, Z koordinatları

            # Vücut keypoints'lerini ekle
            if pose_result.pose_landmarks:
                for lm in pose_result.pose_landmarks.landmark:
                    frame_keypoints.extend([lm.x, lm.y, lm.z])  # X, Y, Z koordinatları

            # Eğer bu karede keypoint bulunduysa diziyi ekle
            if frame_keypoints:
                keypoints_sequence.append(frame_keypoints)

            frame_count += 1  # Frame sayısını artır

    cap.release()

    # JSON formatına ekle
    json_data.append({
        "label": label,
        "keypoints_sequence": keypoints_sequence,
        "frame_count": frame_count
    })

    print(f"{label} etiketi için {frame_count} kare işlendi, {len(keypoints_sequence)} keypoint dizisi kaydedildi.")

def save_json(json_data, output_json_path):
    try:
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=False, indent=4)
        print(f"Veriler {output_json_path} dosyasına kaydedildi.")
    except Exception as e:
        print(f"JSON dosyasına yazılırken hata oluştu: {e}")

# Ana kod: Klasördeki tüm videoları işle ve JSON dosyasını kaydet
def process_videos_in_folder(input_folder, output_json_path):
    json_data = []  # JSON verilerini tutmak için liste

    # Klasördeki tüm video dosyalarını işle
    for file_name in os.listdir(input_folder):
        video_path = os.path.join(input_folder, file_name)
        if not os.path.isfile(video_path):
            continue  # Eğer dosya değilse, atla
        if not file_name.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):  # Video formatları
            continue  # Desteklenmeyen formatları atla

        label = os.path.splitext(file_name)[0]  # Dosya adını etiket olarak kullan
        label = normalize_filename(label)  # Türkçe karakterleri ASCII'ye çevir
        print(f"{label} etiketli video işleniyor...")
        process_video(video_path, label, json_data)

    # JSON verisini kaydet
    save_json(json_data, output_json_path)

# Klasör yollarını tanımlayın
input_folder = "kökdata"  # İşlenecek videoların bulunduğu klasör
output_json_path = "frame_data.json"  # JSON dosyasının kaydedileceği dosya

# Tüm videoları işle
process_videos_in_folder(input_folder, output_json_path)

Anadolu etiketli video işleniyor...
Anadolu etiketi için 88 kare işlendi, 88 keypoint dizisi kaydedildi.
Araba etiketli video işleniyor...
Araba etiketi için 86 kare işlendi, 86 keypoint dizisi kaydedildi.
Arasinda etiketli video işleniyor...
Arasinda etiketi için 123 kare işlendi, 123 keypoint dizisi kaydedildi.
Artmak etiketli video işleniyor...
Artmak etiketi için 103 kare işlendi, 103 keypoint dizisi kaydedildi.
Avrupa etiketli video işleniyor...
Avrupa etiketi için 85 kare işlendi, 85 keypoint dizisi kaydedildi.
Aciklamada etiketli video işleniyor...
Aciklamada etiketi için 112 kare işlendi, 112 keypoint dizisi kaydedildi.
Baslamak etiketli video işleniyor...
Baslamak etiketi için 74 kare işlendi, 74 keypoint dizisi kaydedildi.
Belediye etiketli video işleniyor...
Belediye etiketi için 94 kare işlendi, 94 keypoint dizisi kaydedildi.
Belirtmek etiketli video işleniyor...
Belirtmek etiketi için 96 kare işlendi, 96 keypoint dizisi kaydedildi.
Ben etiketli video işleniyor...
Ben etike

In [12]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import joblib
import numpy as np
import json
from collections import Counter

# ===================== VERİ SETİ ===================== #

# Veri yükleme fonksiyonu
def load_dataset_from_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    X = []  # Keypoint dizileri
    y = []  # Etiketler

    for label, value in raw_data.items():
        keypoints_sequence = value.get("keypoints_sequence", [])
        if len(keypoints_sequence) > 0:
            X.append(keypoints_sequence)
            y.append(label)

    return X, y

# JSON verisini yükle
X, y = load_dataset_from_json("output_data.json")

# Az örnekli sınıfları kontrol et ve filtrele
min_samples = 2  # Her sınıf için minimum örnek sayısı
class_counts = Counter(y)

X_filtered = []
y_filtered = []
for x, label in zip(X, y):
    if class_counts[label] >= min_samples:  # Yeterli sayıda örneğe sahip sınıfları tut
        X_filtered.append(x)
        y_filtered.append(label)

# Veri setini güncelle
X = X_filtered
y = y_filtered

# Etiketleri encode et (String -> Integer)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Veriyi eğitim ve test için böl
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Tensor veri seti sınıfı
class KeypointDataset(Dataset):
    def __init__(self, keypoints, labels):
        self.keypoints = keypoints
        self.labels = labels

    def __len__(self):
        return len(self.keypoints)

    def __getitem__(self, idx):
        sequence = np.array(self.keypoints[idx], dtype=np.float32)
        label = self.labels[idx]
        return torch.tensor(sequence), torch.tensor(label, dtype=torch.long)

# Dataset ve DataLoader
train_dataset = KeypointDataset(X_train, y_train)
test_dataset = KeypointDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# ===================== LSTM MODEL ===================== #

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)  # LSTM'nin son gizli durumu
        out = self.fc(h_n[-1])      # Son gizli durumdan sınıf tahmini
        return out

# Model parametreleri
input_size = 63  # Her frame için keypoints (21 landmark * 3 koordinat)
hidden_size = 128  # LSTM gizli birim boyutu
num_classes = len(le.classes_)  # Sınıf sayısı

model = LSTMModel(input_size, hidden_size, num_classes)

# ===================== EĞİTİM ===================== #

criterion = nn.CrossEntropyLoss()  # Kayıp fonksiyonu
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam optimizasyonu

epochs = 50
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)  # Model tahmini
        loss = criterion(outputs, y_batch)  # Kayıp hesaplama
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Doğruluk hesaplama
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

    train_accuracy = correct / total * 100
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

# Eğitim tamamlandıktan sonra modeli kaydet
torch.save(model.state_dict(), "lstm_model.pth")
joblib.dump(le, "label_encoder.pkl")
print("Model ve LabelEncoder kaydedildi.")

# ===================== TEST ===================== #

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

test_accuracy = correct / total * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")

AttributeError: 'list' object has no attribute 'get'

In [6]:
import cv2
import torch
import numpy as np
import mediapipe as mp
from sklearn.preprocessing import LabelEncoder
import joblib

# MediaPipe eller modülü tanımlaması
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,  # Video akışı için statik görüntü modunda değil
    max_num_hands=1,          # En fazla algılanacak el sayısı
    min_detection_confidence=0.5,  # Minimum algılama güven eşiği
    min_tracking_confidence=0.5    # Minimum takip güven eşiği
)
mp_draw = mp.solutions.drawing_utils

# Model ve Label Encoder yükle
input_size = 63
le = joblib.load("label_encoder.pkl")  # Etiket dönüştürücü (LabelEncoder)
num_classes = len(le.classes_)  # Sınıf sayısını belirle
model = SimpleMLP(input_size, num_classes)  # Modeli yeniden oluştur
model.load_state_dict(torch.load("model.pth"))  # Ağırlıkları yükle
model.eval()  # Modeli değerlendirme moduna al

def extract_keypoints(results):
    if results.multi_hand_landmarks:
        hand = results.multi_hand_landmarks[0]
        keypoints = []
        for lm in hand.landmark:
            keypoints.extend([lm.x, lm.y, lm.z])
        return keypoints
    else:
        return None

cap = cv2.VideoCapture(0)  # Kamera açılır

if not cap.isOpened():
    print("Kamera açılamıyor. Lütfen cihazınızın kamerasını kontrol edin.")
    exit()  # Programdan çıkış

sequence = []
seq_length = 30
predicted_label = ""

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Kamera akışı sağlanamadı. Program kapatılıyor...")
            break

        # Görüntüyü ters çevir ve işle
        image = cv2.flip(frame, 1)
        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)  # MediaPipe işlemi başlat

        if results.multi_hand_landmarks:
            mp_draw.draw_landmarks(image, results.multi_hand_landmarks[0], mp_hands.HAND_CONNECTIONS)
            keypoints = extract_keypoints(results)

            if keypoints:
                sequence.append(keypoints)
                if len(sequence) > seq_length:
                    sequence.pop(0)

                if len(sequence) == seq_length:
                    # Modele uygun forma getir
                    input_tensor = torch.tensor(sequence[-1], dtype=torch.float32).view(1, -1)  # Tensor boyutunu düzelt
                    with torch.no_grad():
                        prediction = model(input_tensor)
                        predicted_class = torch.argmax(prediction, dim=1).item()
                        predicted_label = le.inverse_transform([predicted_class])[0]

        # Ekrana tahmini yaz
        cv2.putText(image, f"Tahmin: {predicted_label}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        cv2.imshow("Tahmin", image)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # 'q' ile çıkış kontrolü
            print("Program kapatılıyor...")
            break
except Exception as e:
    print(f"Bir hata oluştu: {e}")
finally:
    cap.release()  # Kamera serbest bırakılır
    cv2.destroyAllWindows()  # Tüm pencereler kapatılır
    hands.close()  # MediaPipe elleri kapatılır

Program kapatılıyor...
